# Assignment: Segmenting and Clustering Neighborhoods in Toronto

Description: In this assignment, you will be required to <b>explore, segment, and cluster</b> the neighborhoods in the city of Toronto based on the postalcode and borough information.

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

## Problem 1

First, we retreive the URL and creating a Beautiful soup object.

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
soup = BeautifulSoup(source, 'html.parser')

Then we create a list called <b>table_contents</b>. 

And after finding the table and table data,  we create a dictionary called <b>cell</b> that having 3 keys: <b>PostalCode, Borough and Neighborhood</b>.

In [3]:
table_contents=[]

table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        #As postal code contains upto 3 characters extract that using tablerow.p.text
        cell['PostalCode'] = row.p.text[:3]
        #Use split ,strip and replace() for getting Borough and Neighborhood information.
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        #Append to the list
        table_contents.append(cell)
#print(table_contents)

Finally, we create a dataframe with list.

In [4]:
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                     'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                     'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                     'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [5]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [6]:
df.shape

(103, 3)

## Problem 2

First of all, we have to open the geographical file from the url.

In [7]:
url = "https://cocl.us/Geospatial_data"
df1 = pd.read_csv(url)

In [8]:
df1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In order to merge these two dataframe, we have to modify the column name "Postal Code" to "PostalCode" so that they have the same column name for reference.

In [9]:
df1.rename(columns={'Postal Code':'PostalCode'},inplace=True)

In the end, we merge these two dataframe using the merge() funtion.

In [10]:
df_merge = pd.merge(df, df1, on='PostalCode')

In [11]:
df_merge.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [12]:
df_merge.shape

(103, 5)